In [1]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install scikit-optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 4.3 MB/s eta 0:00:00


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from skopt import gp_minimize


<ipython-input-3-318cc7f80aa0>:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
# Directorio donde se encuentran las imágenes originales
original_dir = '/content/drive/MyDrive/s11/Lemon_quality'

# Directorio donde se almacenarán las imágenes divididas en conjuntos de entrenamiento, validación y prueba
base_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset'
os.makedirs(base_dir, exist_ok=True)

# Directorios para las categorías de limones
categories = ['good_quality', 'bad_quality']


In [4]:
# Crear directorios para entrenamiento, validación y prueba
for category in categories:
    os.makedirs(os.path.join(base_dir, 'train', category), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'validation', category), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'test', category), exist_ok=True)

# Copiar imágenes a los directorios correspondientes
for category in categories:
    category_dir = os.path.join(original_dir, category)
    file_list = os.listdir(category_dir)

    # Dividir las imágenes en conjuntos de entrenamiento (80%), validación (10%) y prueba (10%)
    train_files, test_files = train_test_split(file_list, test_size=0.2, random_state=42)
    valid_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

    # Copiar archivos a los directorios correspondientes
    for file_name in train_files:
        src = os.path.join(category_dir, file_name)
        dst = os.path.join(base_dir, 'train', category, file_name)
        shutil.copyfile(src, dst)

    for file_name in valid_files:
        src = os.path.join(category_dir, file_name)
        dst = os.path.join(base_dir, 'validation', category, file_name)
        shutil.copyfile(src, dst)

    for file_name in test_files:
        src = os.path.join(category_dir, file_name)
        dst = os.path.join(base_dir, 'test', category, file_name)
        shutil.copyfile(src, dst)

# Imprimir la cantidad de imágenes en cada conjunto
for split in ['train', 'validation', 'test']:
    for category in categories:
        category_dir = os.path.join(base_dir, split, category)
        print(f"{split} {category} images: {len(os.listdir(category_dir))}")


In [ ]:
import os
import shutil
from PIL import Image

def load_image(file_path):
    try:
        img = Image.open(file_path)
        img.load()
        return img
    except:
        return None

def remove_invalid_images(directory):
    invalid_images = []
    # Recorre todas las imágenes en el directorio
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            # Intenta cargar la imagen
            img = load_image(file_path)
            # Si la imagen no se puede abrir o no existe, la agregamos a la lista de imágenes inválidas
            if img is None:
                invalid_images.append(file_path)

    # Elimina las imágenes inválidas
    for file_path in invalid_images:
        os.remove(file_path)
        print(f"Imagen eliminada: {file_path}")


In [ ]:
# Directorio donde se encuentran las imágenes
train_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/train'
test_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/test'
validation_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/validation'

# Elimina las imágenes inválidas del directorio de entrenamiento
remove_invalid_images(train_dir)
remove_invalid_images(test_dir)
remove_invalid_images(validation_dir)


In [20]:
# Directorios de los conjuntos de datos
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Configuración del generador de imágenes
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generadores de imágenes
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')


Found 1892 images belonging to 2 classes.
Found 297 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [21]:
import os

# Directorio donde se encuentran las divisiones del conjunto de datos
train_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/train'
valid_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/validation'
test_dir = '/content/drive/MyDrive/s11/Lemon_quality_dataset/test'

# Ruta de la imagen eliminada
deleted_image_path = '/content/drive/MyDrive/s11/Lemon_quality_dataset/train/bad_quality/bad_quality_119.jpg'

# Función para verificar la presencia de la imagen en una carpeta
def check_image_presence(directory, image_path):
    for root, dirs, files in os.walk(directory):
        if image_path in files:
            return True
    return False

# Verificar la presencia de la imagen eliminada en cada división del conjunto de datos
train_image_present = check_image_presence(train_dir, deleted_image_path)
valid_image_present = check_image_presence(valid_dir, deleted_image_path)
test_image_present = check_image_presence(test_dir, deleted_image_path)

# Imprimir los resultados
print(f"La imagen eliminada está presente en el conjunto de entrenamiento: {train_image_present}")
print(f"La imagen eliminada está presente en el conjunto de validación: {valid_image_present}")
print(f"La imagen eliminada está presente en el conjunto de prueba: {test_image_present}")


La imagen eliminada está presente en el conjunto de entrenamiento: False
La imagen eliminada está presente en el conjunto de validación: False
La imagen eliminada está presente en el conjunto de prueba: False


In [22]:
# Definir la estructura del modelo
def build_model(hp):
    model = Sequential([
        Conv2D(hp.Int('conv1_filter', min_value=32, max_value=128, step=16), (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv2_filter', min_value=32, max_value=64, step=16), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv3_filter', min_value=32, max_value=64, step=16), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(hp.Int('dense_units', min_value=32, max_value=128, step=16), activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Definir el tuner para la optimización de hiperparámetros
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner',
    project_name='lemon_quality')

# Ejecutar la búsqueda de hiperparámetros
tuner.search(train_generator,
             validation_data=valid_generator,
             epochs=5,
             verbose=1)

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Imprimir el resumen del mejor modelo
best_model.summary()

Trial 5 Complete [00h 01m 02s]
val_accuracy: 0.9562289714813232

Best val_accuracy So Far: 0.9797979593276978
Total elapsed time: 00h 38m 31s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 48)        13872     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 48)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64